## Two Phase Simplex Method
Challenge: Create a Constrained Linear Optimisation algorithm using the two phase simplex method. Test using canonical problem starting from infeasible point. Minimises

In [ ]:
# State canonical form of problem

A_model = [[2,3],
           [16/3,4],
           [2/3,2]]

b_model = [30,64,16]
b_constraints = ['<=','<=','<=']
c_model = [120, 150]

In [ ]:
import numpy as np
import pandas as pd
import os

#######################################################################################################
#%%
# Function declarations

#######################################################################################################

# Intitial processing - setting model format

def switch_constraints(constraint):
    if constraint == "<=":
        return ">="
    elif constraint == ">=":
        return "<="
    else:
        return constraint

def make_b_positive(A, b, b_constraints):
    # turn right side into positive where necessary
    for bi in range(b.shape[0]):
        if b[bi]<0:
            A[bi,:] = -A[bi,:]
            b[bi] = -b[bi]
            b_constraints[bi] = switch_constraints(b_constraints[bi])
    return A, b, b_constraints

def make_Ax_less_b(A,b,b_constraints):
    for bc in range(b.shape[0]):
        if b_constraints[bc] == '>=':
            A[bc,:] = -A[bc,:]
            b[bc] = -b[bc]
            b_constraints[bc] = switch_constraints(b_constraints[bc])
    return A, b, b_constraints

def insert_trivial_constraints(A,b,b_constraints):
    num_main_vars = A.shape[1]
    trivs = np.identity(num_main_vars)
    A = np.vstack(np.vstack((A, trivs)))
    for triv in trivs:
        b_constraints.append('>=')
        b = np.append(b,0)
    return A,b,b_constraints

def insert_modelling_variables(A, b_constraints, c):
    num_main_vars = A.shape[1]
    sur_vars = []
    art_vars = []
    for bc in range(len(b_constraints)):
        s_vec = np.zeros((len(b_constraints)))
        if (b_constraints[bc] == '<='):
            s_vec[bc] = 1
        elif b_constraints[bc] == '>=':
            s_vec[bc] = -1
            sur_vars.append(bc)
        elif b_constraints[bc] == '=':
            s_vec[bc] = 1
            art_vars.append(bc)
            
        A = np.vstack((A.T,s_vec)).T
    c = np.append(c, np.zeros((len(b_constraints))))
    return A, c, np.asarray(sur_vars), np.asarray(art_vars)


def equality_to_inequalities(A,b,b_constraints):
    for bc in range(b.shape[0]):
        if b_constraints[bc] == '=':
            Arow = A[bc,:]
            A = np.vstack((A, -Arow))
            b_constraints[bc] = '<='
            b_constraints = np.append(b_constraints,'>=')
            b = np.append(b,-b[bc])
    return A, b , b_constraints


def to_canonical(A, b, b_constraints, c):
    A, b, b_constraints = make_b_positive(A, b, b_constraints)
    A, c, sur_vars, art_vars = insert_modelling_variables(A, b_constraints, c)
    return A, b, b_constraints, c, sur_vars, art_vars

def to_standard(A, b, b_constraints, c):
    # Turn genaral model into standard model
    # Step 1 - add trivial constrants
    A,b,b_constraints = insert_trivial_constraints(A,b,b_constraints)

    # Step 2 - Make b positive
    A, b, b_constraints = make_Ax_less_b(A,b,b_constraints)
    
    # Step 3 - add 2 inequalities for 1 equality
    A, b , b_constraints = equality_to_inequalities(A,b,b_constraints)
    
    # Step 4 - add slack and surplus variables
    A, c = insert_modelling_variables(A, b_constraints, c)
    return A, b, b_constraints, c

############################################################################################
# Simplex process functions
############################################################################################
# General

def z_func(c, X):
    return np.dot(c,X)

def get_full_X(basic, Xb, full_vars):
    X = np.zeros((full_vars.shape[0]))
    for ba in range(basic.shape[0]):
        X[basic[ba]] = Xb[ba]
    return X

def check_feasibility(A, X, b, b_constraints):
    bres = np.dot(A,X)-b
    feasible = []
    for bc in range(len(b_constraints)):
        if ( b_constraints[bc] == '=' ) and ( bres[bc] == 0 ):
            feasible.append(1)
        elif ( b_constraints[bc] == '<=' ) and ( bres[bc] <= 0 ):
            feasible.append(1)
        elif ( b_constraints[bc] == '>=' ) and ( bres[bc] >= 0 ):
            feasible.append(1)
        else:
            feasible.append(0)
    cons = np.asarray(feasible)
    trivs = (X>=0)*1
    infeasible = (cons==0).any() or (trivs==0 ).any()
    return infeasible, cons,trivs

def construct_p_phase_one(b_constraints):
    p = []
    for bc in b_constraints:
        if bc == '<=':
            p.append(0)
        if bc == '>=':
            p.append(1)
        if bc == '=':
            p.append(-1)
    return np.asarray(p)

############################################################################################
# Phase One functions

def get_basis_matrices_phase_one(A, b, basic, not_basic):
    print("Getting Basis and Non-Basis Matrices and price vectors")
    B = A[:, basic]
    N = A[:, not_basic]
    BInv = np.linalg.inv(B)
    Xb = np.dot(BInv,b)
    return B, BInv, Xb, N

def artificials_feasibility_phase_one(Xb, p):
    # check if vars are feasible
    p_arts = (p==-1)
    p_surs = (p==1)
    Xb_art_feas = (Xb==0)
    Xb_sur_feas = (Xb>=0)
    feasible_arts = np.where((p_arts * Xb_art_feas) == True)[0]
    feasible_surs =  np.where((p_surs * Xb_sur_feas) == True)[0]
    print(f"Feasible artificials: {feasible_arts}")
    print(f"Feasible surplus: {feasible_surs}")
    print()
    return p_arts

def new_basic_and_nonbasic_variables_phase_one(A, Xb, p, BInv, N, basic, not_basic, infeasible):
    # reduced costs
    w = np.dot(p,BInv)
    d = np.dot(w,N)
    print(d)
    dlow_ind = d.argmin()
    if (d.min()>=0) and infeasible:
        print("Problem is infeasible")
        return None
    to_basic = not_basic[dlow_ind]
    print(f"Variable to become basic is: {to_basic}")
    print()
    
    # find first var to become non_basic
    y_0 = np.dot(BInv,A[:,to_basic])
    thetas = Xb/y_0
    to_notbasic_ind = np.where(thetas > 0, thetas, np.inf).argmin()
    to_notbasic = basic[to_notbasic_ind]
    return to_basic, to_notbasic, to_notbasic_ind, dlow_ind

def update_basis_list_phase_one(A, c, p, p_arts, basic, not_basic, to_basic, to_notbasic, dlow_ind, to_notbasic_ind, full_vars, num_main_vars):
    arts_basic = np.where(p_arts == True)[0]+num_main_vars
    try:
        arts_is_feasible = np.isin(arts_basic, to_notbasic)[0]
    except:
        arts_is_feasible = False
    if arts_is_feasible:
        print(f"Found feasible artificial variable: {to_notbasic}")
        # remove art from problem
        full_vars = full_vars[np.where(full_vars != to_notbasic)[0]]
        A = np.delete(A, to_notbasic, axis =1)
        c = np.delete(c, to_notbasic)
        basic[to_notbasic_ind] = to_basic
        not_basic = not_basic[not_basic != to_basic]
        p[to_notbasic-num_main_vars] = 0
        for i in range(not_basic.shape[0]):
            if not_basic[i]>to_notbasic:
                not_basic[i] = not_basic[i]+1
        for i in range(basic.shape[0]):
            if basic[i]>to_notbasic:
                basic[i] = basic[i]+1
    else:
        basic[to_notbasic_ind] = to_basic
        not_basic[dlow_ind] = to_notbasic
    
    return A, c, p, basic, not_basic, full_vars

############################################################################################
# Phase Two functions

def get_basis_matrices_phase_two(A, b, c, X, basic, not_basic):
    print("Getting Basis and Non-Basis Matrices and price vectors")
    B = A[:, basic]
    BInv = np.linalg.inv(B)
    N = A[:, not_basic]
    cb = c[basic]
    cn = c[not_basic]
    Xb = np.dot(BInv,b)
    Xn = X[not_basic]
    return not_basic, B, N, cb, cn, Xb, Xn, BInv
    
def reduced_costs_phase_two(BInv, N,cb, cn, not_basic, solution_found = 0):
    w = np.dot(cb,BInv)
    d = np.dot(w,N) - cn
    dlow = d.min()
    if dlow >=0:
        print("Solution found")
        solution_found = 1
    print(f"Reduced Costs: {d}")
    to_basic_ind = d.argmin()
    to_basic = not_basic[to_basic_ind]
    return to_basic, dlow, d, solution_found

def first_active_constraint_phase_two(basic, to_basic, BInv, A, Xb, full_vars):
    y_q = np.dot(BInv, A[:,to_basic])
    slacks = Xb/y_q
    to_notbasic_ind = np.where(slacks > 0, slacks, np.inf).argmin()
    to_notbasic = basic[to_notbasic_ind]
    basic[ basic == to_notbasic] = to_basic
    not_basic = np.setdiff1d(full_vars, basic)
    print(f"Variable to become not basic is: {to_notbasic}")
    return basic, not_basic

####################################################################################################
#%%

def simplex_two_phase(A,b,b_constraints, c):
    
    # Initial processing
    num_main_vars = A.shape[1]
    A, b, b_constraints, c, sur_vars, art_vars = to_canonical(A, b, b_constraints, c)    
    # Phase One
    # Initial parameters
    # set parameters and basic list for first iteration
    main_vars = np.array([i for i in range(num_main_vars)])
    full_vars = np.array([i for i in range(c.shape[0])])
    basic = np.setdiff1d(full_vars, main_vars)
    not_basic = np.setdiff1d(full_vars, basic)
    
    # get basis matrices and vectors
    X = np.zeros((A.shape[1])) # all_slack
    infeasible, feasC, feasT = check_feasibility(A, X, b, b_constraints)
    
    # if all slack is unfeasible, perform phase 1
    p = construct_p_phase_one(b_constraints)
    Xstar = np.append(main_vars, np.where(p == 0)[0]+num_main_vars)
    
    if not infeasible:
        print()
        print("All-Slack point is not feasible. Starting Phase One to find feasible starting point")
        print()
        
    # start phase 1
    iteration = 1
    while infeasible:
        print("Starting point infeasible, starting pseudo-objective function")
        # check for problems with loop break. Currently, the loop continues to find the next optimum solution,
        # even after the a feasible solution has been found in step a)
        B, BInv, Xb, N = get_basis_matrices_phase_one(A, b, basic, not_basic)
        infeasible, feasC, feasT = check_feasibility(B, Xb, b, b_constraints)
        print(f"iteration {iteration}")
        print(f"Basis: {basic}")
        print(f"Xb: {Xb}")
        print(f"p: {p}")
        print(f"infeasible: {infeasible}")
        print("feasC, feasT")
        print(feasC, feasT)
        print()
        
        # get artificials position in p
        p_arts = artificials_feasibility_phase_one(Xb, p)
        
        print("Check if feasible:")
        B, BInv, Xb, N = get_basis_matrices_phase_one(A, b, basic, not_basic)
        infeasible, feasC, feasT = check_feasibility(B, Xb, b, b_constraints)
        print(f"infeasible: {infeasible}")
        print("feasC, feasT")
        print(feasC, feasT)
        print()
        if not infeasible:
            print("Feasible starting point found")
            break
        
        # Perform reduced costs and first active constraint to get to_basic and to_notbasic
        to_basic, to_notbasic, to_notbasic_ind, dlow_ind = new_basic_and_nonbasic_variables_phase_one(A, Xb, p, BInv, N, basic, not_basic, infeasible)
    
        # edit basic list with to_basic and to_notbasic
        A, c, p, basic, not_basic, full_vars = update_basis_list_phase_one(A, c, p, p_arts, basic, not_basic, to_basic, to_notbasic, dlow_ind, to_notbasic_ind, full_vars, num_main_vars)
        iteration +=1
        B, BInv, Xb, N = get_basis_matrices_phase_one(A, b, basic, not_basic) 
        #infeasible, feasC, feasT = check_feasibility(B, Xb, b, b_constraints) # remove to close loop at earlier feasible point
        
        
        # State feasible starting point 
        X = get_full_X(basic, Xb, full_vars)
        infeasible, feasC, feasT = check_feasibility(A, X, b, b_constraints)
        print()
        print("Feasible Starting point:")
        print(f"X: {X}")
        print(f"infeasible: {infeasible}")
        print("feasC, feasT") 
        print()
        print()

    # Begin Phase Two
    print("Starting Phase Two")
    # Phase two
    solution_found = 0
    iteration = 1
    while True:
        print(f"Iteration {iteration}")
        # Iteration 1 
        print(f"Basis List is: {basic}")
        print(f"Non-Basis List is: {not_basic}")
        print()
        not_basic, B, N, cb, cn, Xb, Xn, BInv = get_basis_matrices_phase_two(A, b, c, X, basic, not_basic)
        X = get_full_X(basic, Xb, full_vars)
    
        print(f"X is feasible: {(X>=0).all()}")
        print(f"z: {z_func(cb,Xb)}")
        
        # reduced cost
        print("Calculating Reduced Costs")
        to_basic, dlow, d, solution_found = reduced_costs_phase_two(BInv, N,cb, cn, not_basic)
        if solution_found == 0:
            print()
            print(f"Variable, q,  to become basic is {to_basic}")
    
            basic, not_basic = first_active_constraint_phase_two(basic, to_basic, BInv, A, Xb, full_vars)
        else:
            break
        iteration+=1
    
    # get optimiser
    X = get_full_X(basic, Xb, full_vars)
    z = z_func(c,X) 
    print()
    print("The optimiser for this model is: ")
    print(f"X = {X}")
    print()
    print(f"The optimum solution is {z}")
    
    return X, z

In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.optimize import linprog
from scipy.optimize import fmin

# Input linear programming model
A = np.array(A_model)
b = np.array(b_model)
c = np.array(c_model)

# Results
# Manual Simplex algorithm 
X, z = simplex_two_phase(A,b,b_constraints,c)

# scipy results
res_linprog = linprog(-c, A_ub = A, b_ub = b, method = 'interior-point')
print()
print("Scipy results:")
print(res_linprog)


All-Slack point is not feasible. Starting Phase One to find feasible starting point

Starting Phase Two
Iteration 1
Basis List is: [2 3 4]
Non-Basis List is: [0 1]

Getting Basis and Non-Basis Matrices and price vectors
X is feasible: True
z: 0.0
Calculating Reduced Costs
Reduced Costs: [-120. -150.]

Variable, q,  to become basic is 1
Variable to become not basic is: 4
Iteration 2
Basis List is: [2 3 1]
Non-Basis List is: [0 4]

Getting Basis and Non-Basis Matrices and price vectors
X is feasible: True
z: 1200.0
Calculating Reduced Costs
Reduced Costs: [-70.  75.]

Variable, q,  to become basic is 0
Variable to become not basic is: 2
Iteration 3
Basis List is: [0 3 1]
Non-Basis List is: [2 4]

Getting Basis and Non-Basis Matrices and price vectors
X is feasible: True
z: 1620.0
Calculating Reduced Costs
Reduced Costs: [ 70. -30.]

Variable, q,  to become basic is 4
Variable to become not basic is: 3
Iteration 4
Basis List is: [0 4 1]
Non-Basis List is: [2 3]

Getting Basis and Non-Bas

SciPy agrees with Simplex alg